<a href="https://colab.research.google.com/github/Avishek2020/A-to-Z-Resources-for-Students/blob/master/SimpleSentenceBgramLModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import nltk, re, pprint
from difflib import SequenceMatcher, get_close_matches, Differ
from collections import Counter
from nltk import word_tokenize
import string
#nltk.download('stopwords')
from nltk.corpus import stopwords 
import matplotlib.pyplot as plt
import math 
from time import time
import collections
from nltk import bigrams, trigrams

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases,Phraser
nltk.download('punkt')
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import numpy as np
from scipy.stats import entropy
from math import log, e
import pandas as pd

import timeit
import inflect
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Calculates the perplexity of the given text.This is simply 2 ** cross-entropy for the text.
def perplexity(entropy):
    #entropy = entropy(text)
    #print(entropy)
    return pow(2.0, entropy)

# entropy
def entropy4(labels, base=None):
  value,counts = np.unique(labels, return_counts=True)
  norm_counts = counts / counts.sum()
  base = e if base is None else base
  return -(norm_counts * np.log(norm_counts)/np.log(base)).sum()

In [4]:
data = []   
sentences = nltk.data.load('/content/gdrive/My Drive/ProjectTextAnalysis/parawiki_english05' , format='raw')
for line in (sentences.decode().split('\n')):
    #print(line)    
    data.append(line)
df = pd.DataFrame(data)
df_new =(df[0].str.split('\t',expand=True,))
df_new.columns = ['Complex Sentence', 'Simple Sentence','Similarity']
df_new.head(5)

,Complex Sentence,Simple Sentence,Similarity
0,"Following the revolution , he organized the fi...",Following the revolution he organized the firs...,0.999061161113
1,Other nearby towns include Haywards Heath to t...,Other nearby towns include Haywards Heath to t...,0.998915119114
2,The project comprised Mei Foo Interchange ( mo...,The project comprised Mei Foo Interchange ( mo...,0.998909131627
3,Pepsi encouraged price-watching consumers to s...,Pepsi encouraged price-watching consumers to s...,0.998829368955
4,"In Britain , musical comedian Mitch Benn has p...","In Britain , musical comedian Mitch Benn has p...",0.998813737193


## Simple Sentence Part

In [5]:
df_new_ss = df_new['Simple Sentence']
len(df_new_ss) 


492994

In [8]:
#split sentences into training and testing sets
from math import floor
def get_training_and_testing_sets(file_list):
    split = 0.7
    split_index = floor(len(file_list) * split)
    training = file_list[:split_index]
    testing = file_list[split_index:]
    return training, testing

#
training_ss, testing_ss = get_training_and_testing_sets(df_new_ss)
print(f" | SIMPLE SENTENCES length of training set {len(training_ss)} | length of testing set {len(testing_ss)} | Total length = {len(training_ss)+len(testing_ss)}")


 | SIMPLE SENTENCES length of training set 345095 | length of testing set 147899 | Total length = 492994


In [9]:
print(training_ss[0])

Following the revolution he organized the first election process in Egyptian universities to choose his successor as Dean of the Faculty in March 2011 , as he left the country on 15.05.2011 in the capacity of cultural counselor at the Egyptian Embassy to Berlin and head of educational mission there .


In [10]:
#df_new.shape[0] # no of col#
training_ss.shape[0]

345095

In [11]:
sent_column_ss ="Simple Sentence"
sent_token_ss = []
for i in range(training_ss.shape[0]):
    if df_new[sent_column_ss][i] is not None:
        sentence_ss = df_new[sent_column_ss][i]            
        #print(sentence_ss)
        #print(i)
        # with each sentence a list of tokens in lower is formed:
        sentRowSS= [doc.lower() for doc in sentence_ss.split(" ")]
        sent_token_ss.extend(sentRowSS)
print("Simple Sentence token --",sent_token_ss[:10])
 # Train a Simple Sentence bigram model.
"""
min_count- ignore all words and bigrams with total collected count lower than this.
threshold- represents a threshold for forming the phrases (higher means fewer phrases). A phrase of words `a` and `b` is accepted if
             `(cnt(a, b) - min_count) * N / (cnt(a) * cnt(b)) > threshold`, where `N` is the total vocabulary size.
"""          
print("=== TRAIN SIMPLE SENTENCES BIGRAM MODEL ===")
bigram_ss = Phrases([sent_token_ss], min_count=1, threshold=1) # higher threshold fewer phrases.
#print([phrase for phrase, score in phrases.export_phrases([sent_token])],[score for phrase, score in phrases.export_phrases([sent_token])]) 
# Export the trained model
bigram_mod_ss = Phraser(bigram_ss)
print("=== SIMPLE SENTENCES BIGRAM MODEL TRAINING DONE===")
# Getting all Bigram from sentence with threshold meet
#for phrase, score in phrases.export_phrases([sent_token]):
    #print(u'{0}   {1}'.format(phrase, score)) 


Simple Sentence token -- ['following', 'the', 'revolution', 'he', 'organized', 'the', 'first', 'election', 'process', 'in']
=== TRAIN SIMPLE SENTENCES BIGRAM MODEL ===
=== SIMPLE SENTENCES BIGRAM MODEL TRAINING DONE===


In [12]:
# Lemmatize the sentences.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs_ss = [[lemmatizer.lemmatize(token) for token in sent_token_ss]]

# To get all Bigram tokens
docs_bigm_ss = []
trainPhrases_ss = []
for idx in range(len(docs_ss)):
    for token in bigram_ss[docs_ss[idx]]:
        trainPhrases_ss.append(token)
        if '_' in token:
            #print(token)
            # Token is a bigram, add to document.
            #docs_ss[idx].append(token)
            docs_bigm_ss.append(token)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [13]:
trainPhrases_ss[:10]

['following',
 'the',
 'revolution',
 'he_organized',
 'the_first',
 'election_process',
 'in',
 'egyptian',
 'university',
 'to_choose']

In [14]:
## ENTROPY of Train Model SS
print(f" Simple Sentences LM Entropy before training set {entropy4(sent_token_ss)}   and Perplexity {perplexity(entropy4(sent_token_ss))}")
print(f" Simple Sentences LM Entropy after training set  {entropy4(trainPhrases_ss)}  and Perplexity {perplexity(entropy4(trainPhrases_ss))}")

 Simple Sentences LM Entropy before training set 7.03568702846942   and Perplexity 131.2057402011086
 Simple Sentences LM Entropy after training set  8.760223510629674  and Perplexity 433.6007706563789


##### # Test Data set Useen SS Sentences 30 %[link text](https://)

In [15]:
# Test Data set Useen Sentences 30 %
testing_ss.shape[0]


147899

In [16]:
sent_column_ss ="Simple Sentence"
sent_token_ss_tst = []
for i in list(testing_ss.index.values):
    if df_new[sent_column_ss][i] is not None:
        sentence_ss_tst = df_new[sent_column_ss][i]            
        sentRowSS_tst= [doc.lower() for doc in sentence_ss_tst.split(" ")]
        sent_token_ss_tst.extend(sentRowSS_tst)
print("Simple Sentence test set token --",sent_token_ss_tst[:10])
#        
# Lemmatize the sentences.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs_ss_tst = [[lemmatizer.lemmatize(token) for token in sent_token_ss_tst]]


Simple Sentence test set token -- ['lleida', '(', ',', ',', ')', 'is', 'a', 'city', 'in', 'western']


In [17]:
print(docs_ss_tst[0][:10])

['lleida', '(', ',', ',', ')', 'is', 'a', 'city', 'in', 'western']


In [20]:
# # Passing unseen simple sentences test set to Train Bigram model
docs_bigm_ss_tst = []
trainPhrases_ss_tst = []
for idx in range(len(docs_ss_tst)):
    for token in bigram_ss[docs_ss_tst[idx]]:
        trainPhrases_ss_tst.append(token)
        if '_' in token:
            #print(token)
            # Token is a bigram, add to document.
            #docs_ss[idx].append(token)
            docs_bigm_ss_tst.append(token)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [21]:
## ENTROPY of Test set SS
print(f" Test Data Simple Sentences LM Entropy before prediction(feeding to Model) {entropy4(sent_token_ss_tst)}   and Perplexity {perplexity(entropy4(sent_token_ss_tst))}")
print(f" Test Data Simple Sentences LM Entropy after  prediction(feeding  to Model)  {entropy4(trainPhrases_ss_tst)}  and Perplexity {perplexity(entropy4(trainPhrases_ss_tst))}")

 Test Data Simple Sentences LM Entropy before prediction(feeding to Model) 6.7998721907021675   and Perplexity 111.42060084071281
 Test Data Simple Sentences LM Entropy after  prediction(feeding  to Model)  8.386915400646417  and Perplexity 334.74423531371644


## Complex Sentences part





In [22]:
df_new_cs = df_new['Complex Sentence']
len(df_new_cs)

492994

In [23]:
#
training_cs, testing_cs = get_training_and_testing_sets(df_new_cs)
print(f" | COMPLEX SENTENCES length of training set {len(training_cs)} | length of testing set {len(testing_cs)} | Total length = {len(training_cs)+len(testing_cs)}")


 | COMPLEX SENTENCES length of training set 345095 | length of testing set 147899 | Total length = 492994


In [24]:
print(training_cs[0])
print(training_ss[0])

Following the revolution , he organized the first election process in Egyptian universities to choose his successor as Dean of the Faculty in March 2011 , as he left the country on 15 May 2011 in the capacity of cultural counselor at the Egyptian Embassy to Berlin and head of the educational mission there .
Following the revolution he organized the first election process in Egyptian universities to choose his successor as Dean of the Faculty in March 2011 , as he left the country on 15.05.2011 in the capacity of cultural counselor at the Egyptian Embassy to Berlin and head of educational mission there .


In [144]:
#df_new.shape[0] # no of col#
training_cs.shape[0]

345095

In [25]:
sent_column_cs ="Complex Sentence"
sent_token_cs = []
for i in range(training_cs.shape[0]):
    if df_new[sent_column_cs][i] is not None:
        sentence_cs = df_new[sent_column_cs][i]            
        #print(sentence_ss)
        #print(i)
        # with each sentence a list of tokens in lower is formed:
        sentRowCS= [doc.lower() for doc in sentence_cs.split(" ")]
        sent_token_cs.extend(sentRowCS)
print("Complex Sentence token --",sent_token_cs[:10])
 # Train a Complex Sentence bigram model.
"""
min_count- ignore all words and bigrams with total collected count lower than this.
threshold- represents a threshold for forming the phrases (higher means fewer phrases). A phrase of words `a` and `b` is accepted if
             `(cnt(a, b) - min_count) * N / (cnt(a) * cnt(b)) > threshold`, where `N` is the total vocabulary size.
"""          
print("=== TRAIN SIMPLE SENTENCES BIGRAM MODEL ===")
bigram_cs = Phrases([sent_token_cs], min_count=1, threshold=1) # higher threshold fewer phrases.
#print([phrase for phrase, score in phrases.export_phrases([sent_token])],[score for phrase, score in phrases.export_phrases([sent_token])]) 
# Export the trained model
bigram_mod_cs = Phraser(bigram_cs)
print("=== COMPLEX SENTENCES BIGRAM MODEL TRAINING DONE===")
# Getting all Bigram from sentence with threshold meet
#for phrase, score in phrases.export_phrases([sent_token]):
    #print(u'{0}   {1}'.format(phrase, score)) 

Complex Sentence token -- ['following', 'the', 'revolution', ',', 'he', 'organized', 'the', 'first', 'election', 'process']
=== TRAIN SIMPLE SENTENCES BIGRAM MODEL ===
=== COMPLEX SENTENCES BIGRAM MODEL TRAINING DONE===


In [27]:
# Lemmatize the sentences.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs_cs = [[lemmatizer.lemmatize(token) for token in sent_token_cs]]

# To get all Bigram tokens
docs_bigm_cs = []
trainPhrases_cs = []
for idx in range(len(docs_cs)):
    for token in bigram_cs[docs_cs[idx]]:
        trainPhrases_cs.append(token)
        if '_' in token:
            #print(token)
            # Token is a bigram, add to document.
            #docs_ss[idx].append(token)
            docs_bigm_cs.append(token)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [28]:
trainPhrases_cs[:10]

['following',
 'the',
 'revolution',
 ',',
 'he_organized',
 'the_first',
 'election_process',
 'in',
 'egyptian',
 'university']

In [29]:
## ENTROPY of Train Model SS
print(f" Complex Sentences LM Entropy before training set {entropy4(sent_token_cs)}   and Perplexity {perplexity(entropy4(sent_token_cs))}")
print(f" Complex Sentences LM Entropy after training set  {entropy4(trainPhrases_cs)}  and Perplexity {perplexity(entropy4(trainPhrases_cs))}")

 Complex Sentences LM Entropy before training set 7.172198915138625   and Perplexity 144.2271455503961
 Complex Sentences LM Entropy after training set  8.78316712286554  and Perplexity 440.5515777393934


 ##### Test Data set Useen CS Sentences 30 % 

In [30]:
# Test Data set Useen Sentences 30 %
testing_cs.shape[0]

147899

In [31]:
sent_column_cs ="Complex Sentence"
sent_token_cs_tst = []
for i in list(testing_cs.index.values):
    if df_new[sent_column_cs][i] is not None:
        sentence_cs_tst = df_new[sent_column_cs][i]            
        sentRowCS_tst= [doc.lower() for doc in sentence_cs_tst.split(" ")]
        sent_token_cs_tst.extend(sentRowCS_tst)
print("Complex Sentence test set token --",sent_token_cs_tst[:10])
#        
# Lemmatize the sentences.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs_cs_tst = [[lemmatizer.lemmatize(token) for token in sent_token_cs_tst]]

Complex Sentence test set token -- ['it', 'is', 'the', 'capital', 'city', 'of', 'the', 'province', 'of', 'lleida']


In [32]:
print(docs_cs_tst[0][:10])

['it', 'is', 'the', 'capital', 'city', 'of', 'the', 'province', 'of', 'lleida']


In [34]:
# # Passing unseen Complex sentences test set to Train Bigram model
docs_bigm_cs_tst = []
trainPhrases_cs_tst = []
for idx in range(len(docs_cs_tst)):
    for token in bigram_cs[docs_cs_tst[idx]]:
        trainPhrases_cs_tst.append(token)
        if '_' in token:
            #print(token)
            # Token is a bigram, add to document.
            #docs_ss[idx].append(token)
            docs_bigm_cs_tst.append(token)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [35]:
## ENTROPY of Test set CS
print(f" Test Data Complex Sentences LM Entropy before prediction(feeding to Model) {entropy4(sent_token_cs_tst)}   and Perplexity {perplexity(entropy4(sent_token_cs_tst))}")
print(f" Test Data Complex Sentences LM Entropy after  prediction(feeding  to Model)  {entropy4(trainPhrases_cs_tst)}  and Perplexity {perplexity(entropy4(trainPhrases_cs_tst))}")

 Test Data Complex Sentences LM Entropy before prediction(feeding to Model) 7.092316799411957   and Perplexity 136.45834362623935
 Test Data Complex Sentences LM Entropy after  prediction(feeding  to Model)  8.503500926215906  and Perplexity 362.91828251928234


##### Simple Sentence Trained Model Predicting Unseen test Complex sentences set 

In [36]:
docs_bigm_ssUcs = []
trainPhrases_ssUcs = []
for idx in range(len(docs_cs_tst)):
    for token in bigram_ss[docs_cs_tst[idx]]:
        trainPhrases_ssUcs.append(token)
        if '_' in token:
            #print(token)
            # Token is a bigram, add to document.
            #docs_ss[idx].append(token)
            docs_bigm_ssUcs.append(token)
print(f" Simple Sentence Trained Model Predicting Unseen test Complex sentences set LM Entropy  {entropy4(trainPhrases_ssUcs)}  and Perplexity {perplexity(entropy4(trainPhrases_ssUcs))}")

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


 Simple Sentence Trained Model Predicting Unseen test Complex sentences set LM Entropy  8.55018770127335  and Perplexity 374.85470531424124


##### Complex Sentence Trained Model Predicting Unseen test Simple sentence set 

In [37]:
docs_bigm_csUss = []
trainPhrases_csUss = []
for idx in range(len(docs_ss_tst)):
    for token in bigram_cs[docs_ss_tst[idx]]:
        trainPhrases_csUss.append(token)
        if '_' in token:
            #print(token)
            # Token is a bigram, add to document.
            #docs_ss[idx].append(token)
            docs_bigm_csUss.append(token)
print(f" Complex Sentence Trained Model Predicting Unseen test Simple sentences set LM Entropy  {entropy4(trainPhrases_csUss)}  and Perplexity {perplexity(entropy4(trainPhrases_csUss))}")

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


 Complex Sentence Trained Model Predicting Unseen test Simple sentences set LM Entropy  8.225147324874545  and Perplexity 299.23753155630885
